In [1]:
# Access parent directories
from sys import path
from os.path import abspath
path += [abspath("../"*i) for i in (1, 2, 3)]

In [ ]:
import gym
from QLab import Qptimizer, ReplayMemory, QNetwork
import torch
from torch import nn
import torch.nn.functional as F
import gym
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import deque
import numpy as np
import random
import tqdm

In [4]:
class DeepNetwork(QNetwork):
    def __init__(self):
        super(self).__init__()
        self.convolution1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.convolution2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.convolution3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7)
        self.fc1 = nn.Linear(in_features=1792, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=256)
        self.fc4 = nn.Linear(in_features=256, out_features=32)
        self.fc5 = nn.Linear(in_features=32, out_features=9)
   
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.convolution1(x), 3))
        x = F.relu(F.max_pool2d(self.convolution2(x), 3))
        x = F.relu(F.max_pool2d(self.convolution3(x), 3, 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
class Agent:
	def __init__(self, qnet : QNetwork, learning_rate = 0.01, gamma = 0.999):
		self.gamma = gamma
		self.memory = ReplayMemory(10000)

		self.policy_net = qnet
		self.target_net = qnet.clone()
		self.target_net.copy_from(self.policy_net)
		self.target_net.eval()

		self.optimizer = torch.optim.RMSprop(self.policy_net.parameters(), lr=learning_rate)
		self.optimizer = Qptimizer(
			self.memory,
			self.optimizer,
			self.policy_net,
			self.target_net
		)

	def __call__(self, state):
		action = utilities.select_epsilon(self.policy_net, state.unsqueeze(0), 0.05) #!
		self.perform_action(action)

	def save(self, path = "./net"):
		self.policy_net.save(path)

	def load(self, path = "./net"):
		self.policy_net.load(path)

	def feedback(self, state, action, reward, new_state):
		self.memory.push(state, action, reward, new_state)
		self.optimizer(gamma = self.gamma)

	def train(self, episodes = 1, update_frq = 10, live = False, plot = False):

		score_history = []
		for episode in tqdm(range(episodes)):
			while not env.terminal():
				next(env)
				if live: env.update()
			score_history.append(env.score)
			if episode % update_frq == 0:
				self.target_net.copy_from(self.policy_net)
			if plot:
				self.optimizer.plot_loss_variance()
				self.optimizer.plot_loss()
				utilities.quickplot(score_history, "Score", "./score")

In [ ]:
env = gym.make("MsPacman-v0")
state_size = env.observation_space.shape
action_size = env.action_space.n